In [ ]:
!pip install librosa soundfile scikit-learn openai-whisper streamlit -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install moviepy transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 30.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:0

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving happy.mp3 to happy.mp3


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sad.mp3 to sad.mp3


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.mp3 to test.mp3


In [ ]:
import librosa
import numpy as np
import soundfile as sf
import os
import whisper
from sklearn.svm import SVC
import joblib
from google.colab import files

# Step 2: Upload Audio Files
uploaded = files.upload()

# Step 3: Feature Extraction

def extract_features(audio_path, sr=22050):
    y, sr = librosa.load(audio_path, sr=sr)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    pitch = librosa.piptrack(y=y, sr=sr)[0].mean(axis=0)
    pitch = pitch[:mfccs.shape[1]]
    pitch = np.pad(pitch, (0, mfccs.shape[1] - pitch.shape[0]), 'constant')
    features = np.vstack((mfccs, pitch))
    return features.mean(axis=1)

# Step 4: Model Training and Saving

def train_model(X_train, y_train):
    model = SVC(kernel='linear', probability=True)
    model.fit(X_train, y_train)
    joblib.dump(model, 'emotion_model.pkl')
    return model

def load_model(path='emotion_model.pkl'):
    return joblib.load(path)

# Step 5: Prepare Dataset (Replace with real files)
files_list = ['happy.mp3', 'sad.mp3']  # Replace with uploaded filenames
targets = ['happy', 'sad']

X, y = [], []
for f, label in zip(files_list, targets):
    if os.path.exists(f):
        features = extract_features(f)
        X.append(features)
        y.append(label)

X, y = np.array(X), np.array(y)
model = train_model(X, y)
print("Model trained and saved successfully.")

# Step 6: Load Whisper and Test on Another Audio File
test_file = 'test.mp3'  # Replace with a test file name from upload
if os.path.exists(test_file):
    print("\nRunning prediction on:", test_file)
    model = load_model()
    asr = whisper.load_model("base")

    transcription = asr.transcribe(test_file)
    print("\nTranscribed Text:", transcription['text'])

    test_features = extract_features(test_file).reshape(1, -1)
    prediction = model.predict(test_features)[0]
    print("\nPredicted Emotion:", prediction)
else:
    print("Test file not found. Upload and re-run this block.")

Saving test.mp3 to test (1).mp3
Saving sad.mp3 to sad (1).mp3
Saving happy.mp3 to happy (1).mp3
Model trained and saved successfully.

Running prediction on: test.mp3


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 266MiB/s]



Transcribed Text:  Hmm

Predicted Emotion: happy
